#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 data and use them to predict the outcome of cases from the last 30 days

##Install the package from github using pip

In [ ]:
#This library is only needed for the Cox Regression Model, which is not included in this tutorial
! pip install lifelines

In [1]:
pwd()

'/home/briarmoss/Documents/Boston_311/models'

In [4]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=19999 sha256=de53473995c392398a2cbcb68232c96c54c16d1f9dcd6f2a4025625f6f6947ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-xs1j5d8c/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1
    Uninstalling boston311-0.1:
      Successfully uninstalled boston311-0.1


##Import the Boston311Model class

In [2]:
! pip show boston311

Name: boston311
Version: 0.1.0
Summary: A package for training machine learning models on Boston 311 data
Home-page: https://github.com/mindfulcoder49/Boston_311
Author: Alex Alcivar
Author-email: alex.g.alcivar49@gmail.com
License: UNKNOWN
Location: /home/briarmoss/.local/lib/python3.10/site-packages
Requires: matplotlib, numpy, pandas, scikit-learn, tensorflow
Required-by: 


In [3]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree

2023-06-16 11:07:12.893854: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 11:07:13.370424: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-16 11:07:13.373510: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 11:07:16.500491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##Get latest file URLS and Current Date Ranges

In [4]:
logregmodel = Boston311LogReg()
logregmodel.load('logregproperties.json', 'logreg.h5')

In [5]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")

print(today_datestring, thirty_days_ago_datestring)

2023-06-16 2023-05-17


In [40]:
logregmodel.predict_date_range = {'start': thirty_days_ago_datestring, 'end': today_datestring}

In [41]:
logregmodel_prediction = logregmodel.predict()

Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0]
770/770 [==============================] - 4s 5ms/step


In [42]:
logregmodel_prediction.head()

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,location_street_name,location_zipcode,latitude,longitude,source,survival_time,event,ward_number,survival_time_hours,event_prediction
8186,101004841184,2023-05-18 10:22:00,2023-05-23 10:22:13,NaT,OVERDUE,Open,,Work Hours-Loud Noise Complaints,Public Works Department,Highway Maintenance,...,65-69 Wareham St,2118.0,42.3390,-71.0674,Constituent Call,NaT,0,8,NaN,0.999817
8323,101004860588,2023-05-31 21:31:00,2023-06-02 08:30:00,NaT,OVERDUE,Open,,Sidewalk Repair (Make Safe),Public Works Department,Highway Maintenance,...,28 Walker St,2129.0,42.3791,-71.0672,Citizens Connect App,NaT,0,2,NaN,0.036834
8396,101004844360,2023-05-20 13:35:00,2023-06-08 08:30:00,NaT,OVERDUE,Open,,Abandoned Vehicles,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,346-348 Tremont St,2116.0,42.3482,-71.0665,Citizens Connect App,NaT,0,5,NaN,0.797962
8402,101004861372,2023-06-01 11:10:00,2023-09-29 11:10:11,NaT,ONTIME,Open,,SCH6/6Unsatisfactory Living Conditions,Inspectional Services,Housing,...,19 Centre St,2119.0,42.3299,-71.0929,Constituent Call,NaT,0,9,NaN,0.829286
8523,101004865438,2023-06-04 16:29:00,2023-06-20 08:30:00,NaT,ONTIME,Open,,Sign Repair,Transportation - Traffic Division,Signs & Signals,...,170-174 South St,2130.0,42.3040,-71.1145,Citizens Connect App,NaT,0,11,NaN,0.139146


In [52]:
import pandas as pd

ml_model_name = 'logregmodel'
ml_model_type = 'logistic regression'
ml_model_id = 1

#create a new dataframe with new columns, ml_model_name, ml_model_type, ml_model_id
ml_model_df = pd.DataFrame({'ml_model_name': [ml_model_name], 
                                    'ml_model_type': [ml_model_type],
                                    'id': [ml_model_id],
                                    'ml_model_date': [today_datestring]})




In [56]:
#get new dataframe with just the event_prediction column from the logregmodel_prediction dataframe
logregmodel_prediction_event = logregmodel_prediction[['event_prediction','case_enquiry_id']].copy()
logregmodel_prediction_event['ml_model_id'] = ml_model_id
#add today's date to the dataframe
logregmodel_prediction_event['prediction_date'] = today_datestring
#rename the event_prediction column to prediction
logregmodel_prediction_event.rename(columns={'event_prediction': 'prediction'}, inplace=True)


In [46]:
#remove logregmodel_prediction event_prediction column
logregmodel_cases = logregmodel_prediction.drop('event_prediction', axis=1).copy()

In [47]:
logregmodel_cases.head()

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,precinct,location_street_name,location_zipcode,latitude,longitude,source,survival_time,event,ward_number,survival_time_hours
8186,101004841184,2023-05-18 10:22:00,2023-05-23 10:22:13,NaT,OVERDUE,Open,,Work Hours-Loud Noise Complaints,Public Works Department,Highway Maintenance,...,0801,65-69 Wareham St,2118.0,42.3390,-71.0674,Constituent Call,NaT,0,8,NaN
8323,101004860588,2023-05-31 21:31:00,2023-06-02 08:30:00,NaT,OVERDUE,Open,,Sidewalk Repair (Make Safe),Public Works Department,Highway Maintenance,...,0206,28 Walker St,2129.0,42.3791,-71.0672,Citizens Connect App,NaT,0,2,NaN
8396,101004844360,2023-05-20 13:35:00,2023-06-08 08:30:00,NaT,OVERDUE,Open,,Abandoned Vehicles,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,0501,346-348 Tremont St,2116.0,42.3482,-71.0665,Citizens Connect App,NaT,0,5,NaN
8402,101004861372,2023-06-01 11:10:00,2023-09-29 11:10:11,NaT,ONTIME,Open,,SCH6/6Unsatisfactory Living Conditions,Inspectional Services,Housing,...,0905,19 Centre St,2119.0,42.3299,-71.0929,Constituent Call,NaT,0,9,NaN
8523,101004865438,2023-06-04 16:29:00,2023-06-20 08:30:00,NaT,ONTIME,Open,,Sign Repair,Transportation - Traffic Division,Signs & Signals,...,1110,170-174 South St,2130.0,42.3040,-71.1145,Citizens Connect App,NaT,0,11,NaN


##Save the prediction data

In [48]:
logregmodel_cases.to_csv('311.csv', index=False)


In [57]:

logregmodel_prediction_event.to_csv('311_predictions.csv', index=False)

In [53]:

ml_model_df.to_csv('311_ml_models.csv', index=False)